<style> p { max-width: 500px; } </style>
# [Integers and Floating-Point Numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Integers-and-Floating-Point-Numbers)

The types are exactly what you'd expect.

They take the form: `[U]Int(n)` where n in the number of bits.

Things like `Int8`, `UInt64`, and `Int128` can be expected.

Doesn't seem to be a 256-bit type, but who needs those really?

You've also got `Float(n)` where the bits options are 16, 32, and 64.
That's your standard half, `single`, and `double`.

I'm guessing you'd use a `Float64` 99% of the time.

There's also a `Bool` type for true/false values (it's stated to be eight bits).

Integers will default to the number of bits of the system.
e.g. `typeof(123)` will be `Int32` on a 32-bit system and `Int64` on a 64-bit.

They'll automatically be promoted when needed.

In [ ]:
# Defaults to system size, signed.
println(typeof(123))

# Hexadeciam is handled smartly. 
# This defaults to (unsigned) 8-bit.
println(typeof(0x01))

# Whereas this defaults to 16-bit due to the leading zeros.
println(typeof(0x0001))

# This is too big to fit into a UInt128 and it promoted to a BigInt.
# Very nice.
println(typeof(1000000000000000000000000000000000000000))

# Floats seems to just get given Float64 by default.
println(typeof(0.01))


# Zero
println(typemin(UInt32))

# The expected value.
println(typemin(Int32))

# A very large number indeed.
println(typemax(UInt128))

# Inf
println(typemax(Float64))

# Compliation error. 
#println(typemax(BigInt))


Int64
UInt8
UInt16
BigInt
Float64
0
-2147483648
340282366920938463463374607431768211455
Inf
-2
251
255
65535
-66
BigInt
UInt8
257
Int64
257
Int64
4294967296
Int64
0
UInt64
0


## [Overflow behavior](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Overflow-behavior)

In [ ]:

println(-2)

# An unsigned wrap around (251)
x = -0x05
println(x)

# 255 as expected.
x = -0b0001
println(x)

# Results in 65535 as a result of its promotion to 16-bit.
x = -0b00000000001
println(x)

# No wrap around, results in BigInt with a value of -66.
x = -0x0000000000000000000000000000000000000000000000000000000042
println(x)
println(typeof(x))

# Does not overflow, it is prompted from UInt8 to UInt64.flags.
# Both when doing calculations *and* the variable when adding to itself.
x = 0xFF
println(typeof(x))
println(x + 2)
println(typeof(x + 2))
x = x + 2
println(x)
println(typeof(x))

# This doesn't overflow either.
x = typemax(UInt32)
x = x + 1
println(x)
println(typeof(x))

# This overflows... why though?
# Why not promote to UInt128? 
# Does it depending on the platform size and only do calculations in that???
x = typemax(UInt64)
x = x + 1
println(x)
println(typeof(x))

x = typemax(UInt128)
x = x + 1
println(x)

# Again this doesn't overflow.
x = typemax(Int32)
x = x + 1
println(x)

# This does.
x = typemax(Int64)
x = x + 1
println(x)


-2
251
255
65535
-66
BigInt
UInt8
257
Int64
257
Int64
4294967296
Int64
0
UInt64
0
2147483648
-9223372036854775808


<style> p { max-width: 500px; } </style>
So that's quite.... interesting.

The overflow logic is a little inconsistant, but it's defined at least.

They say you can use `Base.Checked` if you need a return that says when an overflow occurred.

# [Floating-Point Numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Floating-Point-Numbers)

In [1]:
# A Flaat64 is used by default.
x = 1.0
println(typeof(x))

# Supports the standard method of using a redundant dot to indicate a float.
x = 1.
println(typeof(x))

# You can leave off the leading zero, which is nice.
x = .5
println(x)

# The standard e-notation is supported.
x = 25e3
println(x)

# The usual error with floats apply (0.1 + 0.2 ≠ 0.3)
x = 0.1 + 0.2
println(x)

# Use an 'f' rather than an 'e' to do a single precision float (Float32).
x = 0.5f-3
println(x)
println(typeof(x))

# Hex floats are possible with a 'p'
x = 0x4p2
println(x)
println(typeof(x))

# No special syntax for Float16. 
x = Float16(0.2)
println(x)
println(x + 0.1)
println(typeof(x))

# Bitstring shows the binary contexts of a value.
println(bitstring(0.2))

Float64
Float64
0.5
25000.0
0.30000000000000004
0.0005
Float32
16.0
Float64
0.2
0.299951171875
Float16
0011111111001001100110011001100110011001100110011001100110011010


Yeah, I don't think there's any reason to use Float16.

It's slower by default and much less accurate.

Maybe you need to save memory or export it to a GPU?

Maybe support will one day become common place on x86?


<style> p { max-width: 500px; } </style>
The 'f' (single precision) and 'p' (hex float) seem interesting.

But I think most of the time you'd be using a double.

All your usual floating point rules apply (e.g. `NaN ≠ NaN`).

Though `1/0` and `1.0/0` are allowed and equal `Inf` (with sign of dividend preserved).



In [3]:
# Results in `Inf` as you'd expect.
println(1.0)
println(typeof(1.0))
println(1.0/0)
println(typeof(1.0/0))

# ...also results in `Inf` as integer is converted to Float64
println(1)
println(typeof(1))
println(1/0)
println(typeof(1/0))

# Even when done with variables?!
x = 1
println(x)
println(typeof(x))
x = x / 0
println(x)
println(typeof(x))


1.0
Float64
Inf
Float64
1
Int64
Inf
Float64
1
Int64
Inf
Float64


<style> p { max-width: 500px; } </style>
Once again Julia's implict type conversion is just... weird.

Is there a way to constrain the type???

In [8]:
# eps - epsilon - Defaults to Float64
println(eps())

println(eps(Float64))

println(eps(Float32))

println(eps(Float16))

# Can also be used on a float to determine the distance to the next value.
println(eps(0.2))

println(nextfloat(0.2))
println(prevfloat(0.2))


2.220446049250313e-16
2.220446049250313e-16
1.1920929e-7
0.000977
2.7755575615628914e-17
0.20000000000000004
0.19999999999999998


The workings of floats is definitely something I should study further. 

Many excellent links were provided.

<style> p { max-width: 500px; } </style>

## [Arbitrary Precision Arithmetic](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Arbitrary-Precision-Arithmetic)

So there's not just `BigInt`, but also `BigFloat`.

Having both of these built in is really nice.

You have the methods `setprecision` and `setrounding` to set the standard maximums for when the value is irrational.

Let's have a play around.

In [18]:
# Standard initialiser.
x = BigInt(12345678901234567890)
println(x)
println(typeof(x))

# Quoted - probably not needed here.
# Common in languages without native big types.
x = big"123456789012345678901234567890"
println(x)
println(typeof(x))

# Decimal values are always created as double,
# Meaning this time we have a type conversation error.
# The type is a BigFloat but it loses accurate from the intermediate conversation.
x = BigFloat(0.1)
println(x)
println(typeof(x))

# A better quoted definition.
# This is still accurate but interestingly there's an error after the last (79th) decimal place.
x = BigFloat("0.1")
println(x)
println(typeof(x))

# This error can still accumulate.
x = big"0.1" + big"0.2"
println(x)
println(typeof(x))

setprecision(40) do
    println(big"0.1" + big"0.2")
end

setprecision(1024) do
    println(big"0.1" + big"0.2")
end


12345678901234567890
BigInt
123456789012345678901234567890
BigInt
0.1000000000000000055511151231257827021181583404541015625
BigFloat
0.1000000000000000000000000000000000000000000000000000000000000000000000000000002
BigFloat
0.3000000000000000000000000000000000000000000000000000000000000000000000000000017
BigFloat
0.30000000000018
0.3000000000000000000000000000000000000000000000000000000000000000000000000000006477126416320833469039763897100299678337000273327211038767777602626443852371820293478630627604216169920410470459612247697746324210123960020239752832793556081014685332775115966796875


<style> p { max-width: 500px; } </style>

## [Numeric Literal Coefficients](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#man-numeric-literal-coefficients)

Looks like there's some very nice math code:


In [30]:
x = 5
y = 2

# Scalars can go right in front of constants.
println(12x)
println(12x+3y)

# I'm glad the carot is being used as a power symbol, and not xor.
# Nobody needs xor *that* much.
z = x^2

# Apparently you can't use a '+' in string building...
# This comma in `println` seems to work.
println("Z=",z)
println("√z=", √z)

# This implicit multiplicationeven works with variables.
# But... it must come *after* a bracket, not before.
z = (y+1)x
println(z)

# And you can't do this:
#(x+5)(y-2)




60
66
Z=25
√z=5.0
15


<style> p { max-width: 500px; } </style>
Well that should make formulae *so* much easier to write.

This works really well as a mathematical language.

You can just... use a square root sign. Or put a scaler in front of a variable.

Great stuff!

I have heard people complain that it's hard to type these keys, but if you have a compose key (or WinCompose when on Windows) it's dead simple. I guess that makes you more inclined to use them.

One last thing.... the zero and one functions.


In [32]:
println(zero(Float32))

println(zero(Float16))

println(one(BigFloat))

println(one(Int64))

0.0
0.0
1.0
1


<style> p { max-width: 500px; } </style>
Apparently this helps avoid type conversion? 

I'm not sure.

It'll be great when I learn some control flow.

I'm really itching to write some real programs to try out this newly learnt language! 😀

